# Description

 For the project, we will download data automatically from Citibike and store it into a database. 
 
 We will be using all of the data available from Citibike. In order to do that, we will need to download and save aroun 17 GB of data. Due to the large ammount of data, we can't really import data directly into a pandas dataframe, instead, we have two options:
 
 1. Import data in clusters as a dataframe and process them in batch.
 
 2. Import all the data into a SQLite3 Database on the Hard Drive, and process them through SQL. 
 
 I chose the second one due to the flexibility of the database model, since everything is stored localy and it provides the posibility to append at any time without any repercutions. The data can also be queried by other programs (eg. a website)
 
 
 ## 1. Libraries
 
We are importing all the required libraries for the project 

In [1]:
import pylab as pl #plot package
import pandas as pd #powerful dataframe package
import numpy as np # math package
import os #file management package
import sqlite3 # Database management package
import requests #For downloading files
from zipfile import ZipFile 
import re # for filtering text
import numpy as np # For pretty printing
import glob

## 2. Data preparation

In this step we are setting up data to be analysed.

### 2.1 Directory management 

We are creating a directory named data in the root directory.

We create a data folder in order to have a specific place where we do file manipulation and data management.

This is so we don't have all kinds of files in our root directory and it looks pretty :)

In [2]:

# We check if we are in the data folder 
current_dir=os.getcwd()
if 'data' in current_dir:
    os.chdir('..')

# We are setting the home directory to the Jupyter Root directory
HOME_DIR=os.getcwd()
print (HOME_DIR)

# We are saving the location of the data directory                       
DATA_DIR=HOME_DIR+'/data'
print(DATA_DIR)

# We check if the directory exists, if not,                        
try:
    os.makedirs(DATA_DIR)
    print("Creating directory" + DATA_DIR)
except FileExistsError :
        print("Directory {} already exists".format(DATA_DIR))
os.chdir(DATA_DIR)
print("Changing directory to {}".format(DATA_DIR))

/home/jovyan
/home/jovyan/data
Creating directory/home/jovyan/data
Changing directory to /home/jovyan/data


### 2.2 Database setup

We are setting up a database in order to 

In [3]:
# Creating or connecting to the data sqlite db and setting the cursor there

database_connection = sqlite3.connect('ImportedData.db')
database_cursor = database_connection.cursor()

##TODO Describe cursor and why it's there

In [4]:
# Creating a table that traks downloaded files so we don't have to import them again

database_cursor.execute("CREATE TABLE IF NOT EXISTS file_data(file_name TEXT UNIQUE)")
database_connection.commit()
pd.read_sql_query("Select * from file_data",database_connection)

,file_name


### 2.3 Querying the Citibike data 

We are checking the data available on the citibike website. This is hosted on Amazon AwS in the tripdata Bucket.

The citybike data is hosted on amazon bitbucket 

What we do is we fetch the XML from s3, we parse it for files that are marked down inbetween `<Key>`

We filtered the files that start with `2`, so that we don't get any files from Jersey City, since they start with `JC` and all the others start with `2`

In [5]:
# Fetching a file list of all available files on the Citybike data server

s3_url = "https://s3.amazonaws.com/tripdata"

s3_xml_file = requests.get(s3_url).text

files_on_s3 = re.findall (r'<Key>(2.*?zip)</Key>', s3_xml_file) 
files_on_s3.sort()

print ('We have found the following files on S3 ')

print ("\n".join(files_on_s3))   


We have found the following files on S3 
201306-citibike-tripdata.zip
201307-201402-citibike-tripdata.zip
201307-citibike-tripdata.zip
201308-citibike-tripdata.zip
201309-citibike-tripdata.zip
201310-citibike-tripdata.zip
201311-citibike-tripdata.zip
201312-citibike-tripdata.zip
201401-citibike-tripdata.zip
201402-citibike-tripdata.zip
201403-citibike-tripdata.zip
201404-citibike-tripdata.zip
201405-citibike-tripdata.zip
201406-citibike-tripdata.zip
201407-citibike-tripdata.zip
201408-citibike-tripdata.zip
201409-citibike-tripdata.zip
201410-citibike-tripdata.zip
201411-citibike-tripdata.zip
201412-citibike-tripdata.zip
201501-citibike-tripdata.zip
201502-citibike-tripdata.zip
201503-citibike-tripdata.zip
201504-citibike-tripdata.zip
201505-citibike-tripdata.zip
201506-citibike-tripdata.zip
201507-citibike-tripdata.zip
201508-citibike-tripdata.zip
201509-citibike-tripdata.zip
201510-citibike-tripdata.zip
201511-citibike-tripdata.zip
201512-citibike-tripdata.zip
201601-citibike-tripdata

### 2.4 Setup methods for file management

Here we define all the methods we will use when adding data into the database. We have 5 methods one for downloading, one for unzipping, one for checking if the file exists in the database, one for adding the file in the database, and a cleanup 

In [6]:
def download_file(filename):
    delete_files_ending_with('zip')
    download_url = s3_url + '/' + filename
    print("Downloading file: " + download_url)
    file_data = requests.get(download_url)
    open(filename, 'wb').write(file_data.content)
    
def unzipping_file(filename):
    file = ZipFile(filename,'r')
    print("Extracting... ")
    file.extractall()
    unzipped_file = glob.glob('*.csv')
    print("Extracted " + unzipped_file[0] + " from " + filename + ". Deleting archive to save space...")
    delete_files_ending_with('zip')
    return unzipped_file[0]

def delete_files_ending_with(extension):
    for file in glob.glob('*'+ extension):
            os.remove(file)

def file_has_been_added_to_the_database(filename):
    database_cursor.execute("SELECT file_name FROM file_data WHERE file_name = ?",(filename,))
    if database_cursor.fetchone() is None:
        return False
    else:
        return True

def save_file_name_into_database(filename):
    database_cursor.execute("INSERT INTO file_data (file_name) VALUES(?)",(filename,))
    database_connection.commit()

def import_data_from_file_into_database(filename):
    print("Adding "+ filename + "into the database")
    file_data = pd.read_csv(extracted_file, names=['tripduration','starttime','stoptime','start_station_id','start_station_name',
                                      'start_station_latitude', 'start_station_longitude', 'end_station_id',
                                      'end_station_name', 'end_station_latitude', 'end_station_longitude', 'bikeid',
                                      'usertype', 'birth_year', 'gender'], parse_dates = ['starttime','stoptime'], infer_datetime_format=True, header=None, skiprows=[0])
    file_data['starttime'] =  file_data['starttime'].dt.strftime("%Y-%m-%d %H:%M:%S")
    file_data['stoptime'] =  file_data['stoptime'].dt.strftime("%Y-%m-%d %H:%M:%S")
    file_data.to_sql('trip_data', database_connection, if_exists='append')
    database_connection.commit()
    print("Data from "+filename+" added to the database")
    
    

### 2.5 Inserting data into the database

Now we check compare the data we have with whatever is on the server. With the methods from above, we download, extract, save the data into the database, save the file name into the database when it's parsed, and delete necessary files along the way.

In [7]:
file_count = 0

for file in files_on_s3:
    if not file_has_been_added_to_the_database(file):
        try:
            download_file(file)
            extracted_file = unzipping_file(file)
            import_data_from_file_into_database(extracted_file)
            save_file_name_into_database(file)
            delete_files_ending_with('csv')
            file_count += 1 #la final face count  la cate fisiere am adaugat
        except XLRDError:
            print("Close any open files and try again")

print('We imported '+file_count+' files into imported_data')

Extracting... 
Extracted 201306-citibike-tripdata.csv from 201306-citibike-tripdata.zip. Deleting archive to save space...
Adding 201306-citibike-tripdata.csvinto the database
Data from 201306-citibike-tripdata.csv added to the database
Extracting... 
Extracted 2013-09 - Citi Bike trip data.csv from 201307-201402-citibike-tripdata.zip. Deleting archive to save space...
Adding 2013-09 - Citi Bike trip data.csvinto the database
Data from 2013-09 - Citi Bike trip data.csv added to the database
Extracting... 
Extracted 2013-07 - Citi Bike trip data.csv from 201307-citibike-tripdata.zip. Deleting archive to save space...
Adding 2013-07 - Citi Bike trip data.csvinto the database
Data from 2013-07 - Citi Bike trip data.csv added to the database
Extracting... 
Extracted 2013-08 - Citi Bike trip data.csv from 201308-citibike-tripdata.zip. Deleting archive to save space...
Adding 2013-08 - Citi Bike trip data.csvinto the database
Data from 2013-08 - Citi Bike trip data.csv added to the database


Extracting... 
Extracted 201506-citibike-tripdata.csv from 201506-citibike-tripdata.zip. Deleting archive to save space...
Adding 201506-citibike-tripdata.csvinto the database
Data from 201506-citibike-tripdata.csv added to the database
Extracting... 
Extracted 201507-citibike-tripdata.csv from 201507-citibike-tripdata.zip. Deleting archive to save space...
Adding 201507-citibike-tripdata.csvinto the database
Data from 201507-citibike-tripdata.csv added to the database
Extracting... 
Extracted 201508-citibike-tripdata.csv from 201508-citibike-tripdata.zip. Deleting archive to save space...
Adding 201508-citibike-tripdata.csvinto the database
Data from 201508-citibike-tripdata.csv added to the database
Extracting... 
Extracted 201509-citibike-tripdata.csv from 201509-citibike-tripdata.zip. Deleting archive to save space...
Adding 201509-citibike-tripdata.csvinto the database
Data from 201509-citibike-tripdata.csv added to the database
Extracting... 
Extracted 201510-citibike-tripdata.cs

Extracting... 
Extracted 201708-citibike-tripdata.csv from 201708-citibike-tripdata.csv.zip. Deleting archive to save space...
Adding 201708-citibike-tripdata.csvinto the database
Data from 201708-citibike-tripdata.csv added to the database
Extracting... 
Extracted 201709-citibike-tripdata.csv from 201709-citibike-tripdata.csv.zip. Deleting archive to save space...
Adding 201709-citibike-tripdata.csvinto the database
Data from 201709-citibike-tripdata.csv added to the database
Extracting... 
Extracted 201710-citibike-tripdata.csv from 201710-citibike-tripdata.csv.zip. Deleting archive to save space...
Adding 201710-citibike-tripdata.csvinto the database
Data from 201710-citibike-tripdata.csv added to the database
Extracting... 
Extracted 201711-citibike-tripdata.csv from 201711-citibike-tripdata.csv.zip. Deleting archive to save space...
Adding 201711-citibike-tripdata.csvinto the database
Data from 201711-citibike-tripdata.csv added to the database
Extracting... 
Extracted 201712-citi

Data from 201909-citibike-tripdata.csv added to the database
Extracting... 
Extracted 201910-citibike-tripdata.csv from 201910-citibike-tripdata.csv.zip. Deleting archive to save space...
Adding 201910-citibike-tripdata.csvinto the database
Data from 201910-citibike-tripdata.csv added to the database


NameError: name 'files_count' is not defined

In [8]:
sql_to_be_processed = """SELECT bikeid, tripduration, gender,starttime, stoptime ,usertype, birth_year, 
                        cast(strftime('%Y', stoptime) as int) as stop_year,
                        cast(strftime('%Y', stoptime) as int) as start_year,
                         case 
                            when birth_year != 'NaN' 
                                then strftime('%Y','now') - cast(birth_year as int)
                            else null 
                        end age 
                        FROM trip_data""" 
        
pd.read_sql_query(sql_to_be_processed+ " limit 50", database_connection)

,bikeid,tripduration,gender,starttime,stoptime,usertype,birth_year,stop_year,start_year,age
0,19678,695,1,2013-06-01 00:00:01,2013-06-01 00:11:36,Subscriber,1983.0,2013,2013,36.0
1,16649,693,1,2013-06-01 00:00:08,2013-06-01 00:11:41,Subscriber,1984.0,2013,2013,35.0
2,19599,2059,0,2013-06-01 00:00:44,2013-06-01 00:35:03,Customer,NaN,2013,2013,NaN
3,16352,123,1,2013-06-01 00:01:04,2013-06-01 00:03:07,Subscriber,1960.0,2013,2013,59.0
4,15567,1521,1,2013-06-01 00:01:22,2013-06-01 00:26:43,Subscriber,1983.0,2013,2013,36.0
5,18445,2028,0,2013-06-01 00:01:47,2013-06-01 00:35:35,Customer,NaN,2013,2013,NaN
6,15693,2057,1,2013-06-01 00:02:33,2013-06-01 00:36:50,Subscriber,1991.0,2013,2013,28.0
7,16100,369,1,2013-06-01 00:03:29,2013-06-01 00:09:38,Subscriber,1981.0,2013,2013,38.0
8,15234,1829,1,2013-06-01 00:03:47,2013-06-01 00:34:16,Subscriber,1984.0,2013,2013,35.0
9,16400,829,1,2013-06-01 00:04:22,2013-06-01 00:18:11,Subscriber,1987.0,2013,2013,32.0


In [ ]:
database_cursor.execute("CREATE TABLE bike_data as SELECT bikeid " +
                        " , sum(tripduration) as bike_lifecycle" 
                        " , max(stop_year) as last_year_of_use"
                        " , min(start_year) as first_year_of_use"
                        " , count(bikeid) as number_of_uses"
                        " , count(case when gender = '1' then 1 else null end) as number_of_men_per_bike"
                        " , count(case when gender = '2' then 1 else null end) as number_of_women_per_bike"
                        " , count(case when usertype = 'Subscriber' then 1 else null end) as number_subscribers"
                        " , count(case when  age <= 30 then 1 else null end) as under_30"
                        " , count(case when age > 31 and age <= 40 then 1 else null end) as beween_31_40"
                        " , count(case when age > 41 and age <= 50 then 1 else null end) as beween_41_50"
                        " , count(case when age > 50 then 1 else null end) as over_50"
                        " FROM (" + sql_to_be_processed +")" 
                        "GROUP BY bikeid ")

database_connection.commit()









